Création du modèle pour 52 fichiers

Bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import cv2 as cv

#partie sklearn
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#partie tensorFLow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop


In [ ]:
size=128 #taille des images
nbDir=52 #nombre de fichiers de fleurs

Etape 1: Nettoyage des images

In [ ]:
data=[]
label=[]
FlowerPath='../../flower52'

#récupération des images et des noms d'espèces   
os.listdir(FlowerPath)
for folder in os.listdir(FlowerPath):
    for file in os.listdir(os.path.join(FlowerPath,folder)):
        if  file.endswith("jpeg"):
            label.append(folder) #on met le nom des dossiers (des espèces) dans label
            img = cv.imread(os.path.join(FlowerPath,folder, file))
            img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            im = cv.resize(img_rgb, (size,size)) #recadre les images
            data.append(im) #on met les images dans data
                    
        else: 
            continue

data_arr = np.array(data)
label_arr = np.array(label)
y = LabelEncoder().fit_transform(label_arr)
y = to_categorical(y,nbDir)
X = data_arr/255



Etape 2: Séparation des données, 20% dans test et le reste dans train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

Etape 3: Création du modèle

In [ ]:
model = Sequential()
model.add(Conv2D(filters = size/2, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (size,size,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = size/4, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = size/2, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = size, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(size, activation='relu'))
model.add(Dense(size/2, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(nbDir, activation = "softmax"))


Etape 4: Premier entrainement

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=32
epochs=20

checkpoint_path = "modele52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),   
                              verbose = 1,
                              callbacks=[cp_callback])

Etape 5: premier test

In [ ]:
categories = np.sort(os.listdir(FlowerPath))
fig, ax = plt.subplots(4,4, figsize=(25, 40))

for i in range(4):
    for j in range(4):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("VRAI: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTION: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(size, size, 3), cmap='gray')
        else:
            ax[i,j].set_title("VRAI: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTION: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(size, size, 3), cmap='gray')

Etape 6: Sauvegarde du modèle en mémoire

In [ ]:
#model.save('modele52/my_model') #si décommenté écrase le modèle précédant
del model